In [ ]:
import re
import pandas as pd
from google.colab import files
import io
import chardet

# 렉시콘 기반 췌장 영상 판독문 분석기
class LexiconBasedAnalyzer:
    def __init__(self):
        # 위치 패턴
        self.LOCATION_PATTERNS = {
            'head': re.compile(r'\bhead\b|\bhead of pancreas\b|\b두부\b|\b췌두부\b|\b췌장\s*두부\b|\bHOP\b', re.IGNORECASE),
            'body': re.compile(r'\bbody\b|\bbody of pancreas\b|\b체부\b|\b췌체부\b|\b췌장\s*체부\b|\bBOP\b', re.IGNORECASE),
            'tail': re.compile(r'\btail\b|\btail of pancreas\b|\b미부\b|\b췌미부\b|\b췌장\s*미부\b|\bTOP\b', re.IGNORECASE),
            'neck': re.compile(r'\bneck\b|\bneck of pancreas\b|\b경부\b|\b췌경부\b|\b췌장\s*경부\b|\bNOP\b', re.IGNORECASE),
            'uncinate process': re.compile(r'\buncinate\s*process\b|\b구상돌기\b|\b구상 돌기\b|\b췌장\s*구상돌기\b|\bUP\b|\buncinate\b', re.IGNORECASE),
            'whole pancreas': re.compile(r'\bwhole\s*pancreas\b|\bentire\s*pancreas\b|\bdiffuse\s*pancreas\b|\b전체\s*췌장\b|\b췌장\s*전체\b|\b전 췌장\b|\b미만성\s*췌장\b', re.IGNORECASE)
        }

        # 크기 패턴
        self.SIZE_PATTERN = re.compile(r'(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)', re.IGNORECASE)

        # 추가 크기 패턴
        self.ADDITIONAL_SIZE_PATTERNS = [
            re.compile(r'약\s+(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*(?:크기|size))?', re.IGNORECASE),
            re.compile(r'(\d+\.?\d*)(?:mm|cm|㎜|㎝)(?:\s*(?:크기|size))?', re.IGNORECASE),
            re.compile(r'(\d+\.?\d*)[~\-](\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)', re.IGNORECASE),
            re.compile(r'(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*에서\s*)(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*으?로)', re.IGNORECASE)
        ]

        # 낭종(Cyst) 관련 패턴
        self.CYST_PATTERNS = {
            'basic': re.compile(r'\bcyst\b|\bcystic lesion\b|\b낭종\b|\b낭성\s*병변\b', re.IGNORECASE),
            'new': re.compile(r'\bnew(?:ly)?\s*(?:develop\w+|appear\w+)?\s*cyst\b|\b새로운?\s*(?:발견된?)?\s*낭종\b', re.IGNORECASE),
            'increased': re.compile(r'\b(?:cyst|낭종).*?(?:increase[d]?|enlarge[d]?|progression)\b|\b(?:크기)?\s*(?:증가|커진)\s*(?:낭종|낭성\s*병변)\b', re.IGNORECASE),
            'decreased': re.compile(r'\b(?:cyst|낭종).*?(?:decrease[d]?|smaller|regression)\b|\b(?:크기)?\s*(?:감소|작아진)\s*(?:낭종|낭성\s*병변)\b', re.IGNORECASE),
            'stable': re.compile(r'\b(?:cyst|낭종).*?(?:stable|unchanged|no\s*change)\b|\b변화\s*없는\s*(?:낭종|낭성\s*병변)\b', re.IGNORECASE),
            'duct_comm': re.compile(r'\bcyst.*?(?:communicat\w+|connect\w+).*?(?:main|pancreatic)\s*duct\b|\b낭종.*?(?:주|췌)관.*?(?:연결|교통)\b', re.IGNORECASE),
            'bd_ipmn': re.compile(r'\bbranch\s+duct\s+(?:type\s+)?IPMN\b|\bBD-?IPMN\b|\b분지(?:형|형태)?\s*(?:IPMN|관내유두상점액종양?)\b', re.IGNORECASE),
            'md_ipmn': re.compile(r'\bmain\s+duct\s+(?:type\s+)?IPMN\b|\bMD-?IPMN\b|\b주췌관(?:형|형태)?\s*(?:IPMN|관내유두상점액종양?)\b', re.IGNORECASE)
        }

        # 결절(Nodule) 관련 패턴
        self.NODULE_PATTERNS = {
            'basic': re.compile(r'\bnodule\b|\bnodular lesion\b|\b결절\b|\b결절성\s*병변\b', re.IGNORECASE),
            't1_signal': re.compile(r'\bt1[w]?\s*(?:high|hyper|low|hypo|iso)\s*(?:signal|intensity)\b|\bt1.*?(?:고|저|동등)신호\b', re.IGNORECASE),
            't2_signal': re.compile(r'\bt2[w]?\s*(?:high|hyper|low|hypo|iso)\s*(?:signal|intensity)\b|\bt2.*?(?:고|저|동등)신호\b', re.IGNORECASE),
            'diffusion': re.compile(r'\bdiffusion\s*(?:restriction|restricted)\b|\bdwi\s*high\b|\badc\s*low\b|\b확산\s*제한\b', re.IGNORECASE),
            'enhancement': re.compile(r'\b(?:rim|peripheral|homogeneous|heterogeneous)\s*enhancement\b|\b(?:테두리|변연부|균일|불균일)(?:한|하게)?\s*조영증강\b', re.IGNORECASE)
        }

        # 종괴(Mass) 관련 패턴
        self.MASS_PATTERNS = {
            'basic': re.compile(r'\bmass\b|\btumor\b|\b종괴\b|\b종양\b', re.IGNORECASE),
            'large': re.compile(r'\blarge\s*(?:mass|tumor)\b|\bmassive\s*(?:mass|tumor)\b|\bhuge\s*(?:mass|tumor)\b|\b큰\s*(?:종괴|종양)\b|\b거대한\s*(?:종괴|종양)\b', re.IGNORECASE),
            'invasion': re.compile(r'\b(?:mass|tumor|종괴|종양).*?(?:invad\w+|invasive|invasion)\b|\b(?:침윤|침습|침범)(?:하는|된)?\s*(?:종괴|종양)\b', re.IGNORECASE),
            'compression': re.compile(r'\b(?:mass|tumor|종괴|종양).*?(?:compress\w+|compression|mass effect)\b|\b(?:압박|압박효과|전위)(?:하는|된)?\s*(?:종괴|종양)\b', re.IGNORECASE),
            'vessel': re.compile(r'\b(?:mass|tumor|종괴|종양).*?(?:vessel|vascular|artery|vein).*?(?:involvement|encasement|invasion)\b|\b(?:혈관|동맥|정맥).*?(?:침범|침윤|포위)(?:하는|된)?\s*(?:종괴|종양)\b', re.IGNORECASE)
        }

        # 변화 정보 패턴
        self.CHANGE_PATTERN = re.compile(
            r'\b(?:increased?|decreased?|no\s*change|interval\s*change|'
            r'increase[d]?\s*in\s*size|decrease[d]?\s*in\s*size|'
            r'newly\s*appear(?:ed)?|disappeared?|unchanged|stable|'
            r'변화|증가|감소|커짐|작아짐|새롭게|커진|작아진|신생|소실|'
            r'변화\s*없음|이전과\s*동일|이전과\s*같은)\b',
            re.IGNORECASE
        )

        # 신호 강도 패턴
        self.SIGNAL_PATTERN = re.compile(r'\b(?:high|low|iso)(?:\s*(?:signal|intensity|SI))\b|\bt[12][wc]?[i]?\s*(?:high|hyper|low|hypo|iso)(?:\s*(?:intense|intensity|signal))?\b|\bT[12].*?(?:고|저|동등)신호\b', re.IGNORECASE)

        # 복합 병변 패턴
        self.COMPLEX_LESION_TYPES = {
            'septated cystic lesion': [
                r'\bseptat(?:ed|ion|a|ions)\s+(?:cyst|cystic)\b',
                r'\bmulti\s*sept(?:ated|ation|a|ations)\b',
                r'\bseptat(?:ed|ion|a|ions)\b',
                r'\b격벽(?:\s*있는?|성)?\s*(?:낭종|낭성|낭종성|낭성\s*병변)\b'
            ],
            'cystic nodule': [
                r'\bcystic\s+nodule\b',
                r'\bcystic.*?nodular\b',
                r'\bcystic.*?nodule\b',
                r'\bnodule.*?cystic\b',
                r'\b낭성\s*(?:결절|결절성|결절성?\s*병변)\b'
            ],
            'cystic mass': [
                r'\bcystic\s+mass\b',
                r'\bcystic.*?mass\b',
                r'\bmass.*?cystic\b',
                r'\b낭성\s*(?:종괴|종양|종괴성|종양성|병변|종괴성?\s*병변|종양성?\s*병변)\b'
            ],
            'branch duct IPMN': [
                r'\bbranch\s+(?:duct)?\s*(?:type)?\s*(?:IPMN|intraductal\s*papillary\s*mucinous\s*neoplasm)\b',
                r'\bBD\-?IPMN\b',
                r'\b분지형?\s*(?:IPMN|관내유두상점액종양?|관내유두상점액낭종?)\b'
            ],
            'main duct IPMN': [
                r'\bmain\s+(?:duct)?\s*(?:type)?\s*(?:IPMN|intraductal\s*papillary\s*mucinous\s*neoplasm)\b',
                r'\bMD\-?IPMN\b',
                r'\b주췌관형?\s*(?:IPMN|관내유두상점액종양?|관내유두상점액낭종?)\b'
            ]
        }

    def clean_text(self, text):
        """텍스트 정제 함수"""
        if not isinstance(text, str):
            return ""

        # 이스케이프 시퀀스 제거
        text = re.sub(r'_x000D_', '', text)
        text = re.sub(r'\\n', '\n', text)

        # 연속된 공백 제거
        text = re.sub(r'\s+', ' ', text)

        # 여러 줄바꿈을 하나로 통일
        text = re.sub(r'\n\s*\n+', '\n\n', text)

        return text.strip()

    def analyze_report(self, conclusion, report_text=None):
        """판독문 분석 함수"""
        # 결과 초기화
        result = {
            'lesion_presence': 'No',
            'lesion_type': '',
            'complex_lesion_type': '',
            'multiplicity': 'No',
            'location': '',
            'size': '',
            'signal_intensity': '',
            'change': '',
            # 병변 유형별 특화 필드
            'temporal_classification': '',
            'duct_relation': '',
            'mri_signal': '',
            'enhancement_pattern': '',
            'size_description': '',
            'tissue_effect': ''
        }

        # 텍스트 정제
        conclusion = self.clean_text(conclusion)
        if report_text:
            report_text = self.clean_text(report_text)

        # 모든 텍스트 결합
        combined_text = conclusion
        if report_text:
            combined_text += " " + report_text

        # 텍스트가 없으면 기본 결과 반환
        if not combined_text.strip():
            return result

        # 1. 병변 유형 감지
        # 낭종(Cyst) 감지
        cyst_detected = False
        for pattern_name, pattern in self.CYST_PATTERNS.items():
            if pattern.search(combined_text):
                cyst_detected = True
                # 시간적 분류 추출 (new, increased, decreased, stable)
                if pattern_name in ['new', 'increased', 'decreased', 'stable']:
                    result['temporal_classification'] = pattern_name
                # 췌관 관계 추출
                elif pattern_name in ['duct_comm', 'bd_ipmn', 'md_ipmn']:
                    result['duct_relation'] = pattern_name

        # 결절(Nodule) 감지
        nodule_detected = False
        for pattern_name, pattern in self.NODULE_PATTERNS.items():
            if pattern.search(combined_text):
                nodule_detected = True
                # MRI 신호와 조영증강 패턴 추출
                if pattern_name in ['t1_signal', 't2_signal', 'diffusion']:
                    if not result['mri_signal']:
                        result['mri_signal'] = pattern_name
                    else:
                        result['mri_signal'] += ', ' + pattern_name
                elif pattern_name == 'enhancement':
                    match = pattern.search(combined_text)
                    result['enhancement_pattern'] = match.group(0) if match else ''

        # 종괴(Mass) 감지
        mass_detected = False
        for pattern_name, pattern in self.MASS_PATTERNS.items():
            if pattern.search(combined_text):
                mass_detected = True
                # 종괴 특화 정보 추출
                if pattern_name == 'large':
                    result['size_description'] = 'large'
                elif pattern_name in ['invasion', 'compression', 'vessel']:
                    if not result['tissue_effect']:
                        result['tissue_effect'] = pattern_name
                    else:
                        result['tissue_effect'] += ', ' + pattern_name

        # 병변 존재 여부 및 유형 결정
        if cyst_detected or nodule_detected or mass_detected:
            result['lesion_presence'] = 'Yes'

            # 병변 유형 결정 (우선순위: 종괴 > 결절 > 낭종)
            if mass_detected:
                result['lesion_type'] = 'mass'
            elif nodule_detected:
                result['lesion_type'] = 'nodule'
            elif cyst_detected:
                result['lesion_type'] = 'cyst'

        # 2. 복합 병변 유형 감지
        for complex_type, patterns in self.COMPLEX_LESION_TYPES.items():
            for pattern in patterns:
                if re.search(pattern, combined_text, re.IGNORECASE):
                    result['complex_lesion_type'] = complex_type
                    break
            if result['complex_lesion_type']:
                break

        # 3. 위치 정보 추출
        locations = []
        for location, pattern in self.LOCATION_PATTERNS.items():
            if pattern.search(combined_text):
                locations.append(location)

        if locations:
            result['location'] = ', '.join(sorted(set(locations)))

        # 4. 크기 정보 추출
        sizes = []
        # 기본 크기 패턴
        size_matches = self.SIZE_PATTERN.findall(combined_text)
        if size_matches:
            sizes.extend(size_matches)

        # 추가 크기 패턴
        for pattern in self.ADDITIONAL_SIZE_PATTERNS:
            more_sizes = pattern.findall(combined_text)
            if more_sizes:
                # 튜플로 반환되는 경우 (범위 표현 등) 처리
                for match in more_sizes:
                    if isinstance(match, tuple):
                        sizes.extend(match)
                    else:
                        sizes.append(match)

        if sizes:
            # 중복 제거 및 숫자로 정렬
            unique_sizes = sorted(set(sizes), key=lambda x: float(x))
            result['size'] = ', '.join(unique_sizes)

        # 5. 변화 정보 추출
        change_matches = self.CHANGE_PATTERN.findall(combined_text)
        if change_matches:
            # 중복 제거 및 변화 정보 조합
            unique_changes = sorted(set(change_matches))
            result['change'] = ', '.join(unique_changes)

        # 6. 신호 강도 정보 추출
        signal_matches = self.SIGNAL_PATTERN.findall(combined_text)
        if signal_matches:
            # 중복 제거 및 신호 강도 정보 조합
            unique_signals = sorted(set(signal_matches))
            result['signal_intensity'] = ', '.join(unique_signals)

        # 7. 다발성 여부 판단
        # 다발성 관련 키워드 확인
        multiplicity_terms = [
            'multiple', 'several', 'a few', 'few', 'two', 'three', 'four', 'five',
            'many', 'numerous', 'both', 'bilateral', 'multifocal', '다발성', '여러', '몇 개'
        ]

        for term in multiplicity_terms:
            if re.search(r'\b' + term + r'\b', combined_text, re.IGNORECASE):
                result['multiplicity'] = 'Yes'
                break

        # 위치가 2개 이상이면 다발성으로 간주
        if len(set(locations)) >= 2:
            result['multiplicity'] = 'Yes'

        return result

def process_files_with_lexicon_analyzer():
    """렉시콘 기반 분석기로 파일 처리"""
    print("렉시콘 기반 췌장 영상 판독문 분석기를 실행합니다:")

    # 파일 업로드
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return None

    # 첫 번째 업로드된 파일 처리
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    print(f"파일 '{file_name}' 업로드 완료 ({len(file_content)} bytes)")

    # 파일 형식에 따라 데이터프레임 생성
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        df = pd.read_excel(io.BytesIO(file_content))
    else:
        try:
            df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(io.BytesIO(file_content), encoding='cp949')

    print(f"데이터프레임 생성 완료: {len(df)} 행, {len(df.columns)} 열")
    print("컬럼:", df.columns.tolist())

    # 분석기 초기화
    analyzer = LexiconBasedAnalyzer()

    # 결과 처리
    results = []

    for idx, row in df.iterrows():
        try:
            # 필수 필드 안전하게 추출
            patient_id = str(row.get('변환 ID', '')) if '변환 ID' in df.columns else ''
            exam_code = str(row.get('검사코드#5', '')) if '검사코드#5' in df.columns else ''
            exam_date = str(row.get('처방일자#3', '')) if '처방일자#3' in df.columns else ''
            exam_type = str(row.get('검사명#6', '')) if '검사명#6' in df.columns else ''
            report_text = str(row.get('검사결과(text)#7', '')) if '검사결과(text)#7' in df.columns else ''
            conclusion = str(row.get('결론및진단#8', '')) if '결론및진단#8' in df.columns else ''

            # 판독문 분석
            analysis_result = analyzer.analyze_report(conclusion, report_text)

            # 결과 통합
            result = {
                'id': patient_id,
                'exam_code': exam_code,
                'exam_type': exam_type,
                'exam_date': exam_date,
                **analysis_result,
                'conclusion': conclusion[:100] + '...' if len(conclusion) > 100 else conclusion
            }

            results.append(result)

            # 진행상황 표시
            if (idx + 1) % 20 == 0 or idx == len(df) - 1:
                print(f"처리 중: {idx + 1}/{len(df)} 완료")

        except Exception as e:
            print(f"행 {idx} 처리 중 오류 발생: {e}")

    # 결과 데이터프레임 생성
    results_df = pd.DataFrame(results)

    # 결과를 CSV 파일로 저장
    output_filename = f"lexicon_analysis_{file_name.split('.')[0]}.csv"
    results_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"분석 결과가 '{output_filename}' 파일로 저장되었습니다.")

    # 결과 요약
    print("\n분석 결과 요약:")
    print(f"총 처리 건수: {len(results_df)}")
    print(f"병변 감지 건수: {len(results_df[results_df['lesion_presence'] == 'Yes'])}")
    print(f"낭종 감지 건수: {len(results_df[results_df['lesion_type'] == 'cyst'])}")
    print(f"결절 감지 건수: {len(results_df[results_df['lesion_type'] == 'nodule'])}")
    print(f"종괴 감지 건수: {len(results_df[results_df['lesion_type'] == 'mass'])}")

    return results_df

# 분석 실행
lexicon_results = process_files_with_lexicon_analyzer()

렉시콘 기반 췌장 영상 판독문 분석기를 실행합니다:


Saving pancreas_1300data.csv to pancreas_1300data.csv
파일 'pancreas_1300data.csv' 업로드 완료 (1690248 bytes)
데이터프레임 생성 완료: 1302 행, 6 열
컬럼: ['변환 ID', '처방일자#3', '검사코드#5', '검사명#6', '검사결과(text)#7', '결론및진단#8']
처리 중: 20/1302 완료
처리 중: 40/1302 완료
처리 중: 60/1302 완료
처리 중: 80/1302 완료
처리 중: 100/1302 완료
처리 중: 120/1302 완료
처리 중: 140/1302 완료
처리 중: 160/1302 완료
처리 중: 180/1302 완료
처리 중: 200/1302 완료
처리 중: 220/1302 완료
처리 중: 240/1302 완료
처리 중: 260/1302 완료
처리 중: 280/1302 완료
처리 중: 300/1302 완료
처리 중: 320/1302 완료
처리 중: 340/1302 완료
처리 중: 360/1302 완료
처리 중: 380/1302 완료
처리 중: 400/1302 완료
처리 중: 420/1302 완료
처리 중: 440/1302 완료
처리 중: 460/1302 완료
처리 중: 480/1302 완료
처리 중: 500/1302 완료
처리 중: 520/1302 완료
처리 중: 540/1302 완료
처리 중: 560/1302 완료
처리 중: 580/1302 완료
처리 중: 600/1302 완료
처리 중: 620/1302 완료
처리 중: 640/1302 완료
처리 중: 660/1302 완료
처리 중: 680/1302 완료
처리 중: 700/1302 완료
처리 중: 720/1302 완료
처리 중: 740/1302 완료
처리 중: 760/1302 완료
처리 중: 780/1302 완료
처리 중: 800/1302 완료
처리 중: 820/1302 완료
처리 중: 840/1302 완료
처리 중: 860/1302 완료
처리 중: 880/1302 완료
처리 중: 900/13

In [ ]:
import re
import pandas as pd
from google.colab import files
import io
from collections import Counter
import chardet

# 개선된 PancreaticRadiologyAnalyzer 클래스
class PancreaticRadiologyAnalyzer:
    def __init__(self):
        # 위치 패턴 - 한글/영어 표현 모두 포함
        self.LOCATION_PATTERNS = {
            'head': re.compile(r'\bhead\b|\b두부\b|\b췌두부\b|\b췌장\s*두부\b', re.IGNORECASE),
            'body': re.compile(r'\bbody\b|\b체부\b|\b췌체부\b|\b췌장\s*체부\b', re.IGNORECASE),
            'tail': re.compile(r'\btail\b|\b미부\b|\b췌미부\b|\b췌장\s*미부\b', re.IGNORECASE),
            'neck': re.compile(r'\bneck\b|\b경부\b|\b췌경부\b|\b췌장\s*경부\b', re.IGNORECASE),
            'uncinate process': re.compile(r'\buncinate process\b|\b구상돌기\b|\b췌장\s*구상돌기\b', re.IGNORECASE),
            'whole pancreas': re.compile(r'\bwhole pancreas\b|\b췌장\s*전체\b|\b전체\s*췌장\b', re.IGNORECASE)
        }

        # 크기 패턴 - 다양한 표현 방식 포함
        self.SIZE_PATTERN = re.compile(r'(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)', re.IGNORECASE)

        # 추가 크기 패턴
        self.ADDITIONAL_SIZE_PATTERNS = [
            # "약 X cm 크기" 형태
            re.compile(r'약\s+(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*(?:크기|size))?', re.IGNORECASE),
            # "XcmYmm 크기" 형태 (공백 없는 경우)
            re.compile(r'(\d+\.?\d*)(?:mm|cm|㎜|㎝)(?:\s*(?:크기|size))?', re.IGNORECASE),
            # 범위 표현: "X-Y cm" 형태
            re.compile(r'(\d+\.?\d*)[~\-](\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)', re.IGNORECASE),
            # 변화 표현: "Xcm에서 Ycm으로" 형태
            re.compile(r'(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*에서\s*)(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*으?로)', re.IGNORECASE)
        ]

        # 변화 정보 패턴
        self.CHANGE_PATTERN = re.compile(
            r'\b(?:increased|decreased|no change|interval change|increased in size|'
            r'decreased in size|newly appeared|disappeared|resolution|improvement|'
            r'aggravation|progression|변화|증가|감소|커짐|작아짐|새롭게|interval|increase|'
            r'decrease|larger|smaller|slightly|somewhat|mild|moderate|marked)\b',
            re.IGNORECASE
        )

        # 신호 강도 패턴
        self.SIGNAL_PATTERN = re.compile(r'\b(?:high|low|iso)\s*(?:signal|intensity)', re.IGNORECASE)

        # 한글 인코딩 오류 매핑
        self.KOREAN_MAPPINGS = {
            '媛?': '간',
            '鍮꾩옣': '비장',
            '?대궘': '담낭',
            '?좎옣': '신장',
            '?섑옣': '췌장',
            '?앺넻': '흉부',
            '寃고곷': '결과',
            '?뚮Т': '무우',
        }

        # Complex lesion 패턴
        self.COMPLEX_LESION_TYPES = {
            'septated cystic lesion': [
                r'\bseptated\s+cystic\b',
                r'\bmultiseptated\b',
                r'\bseptation\b',
                r'\bseptated\b'
            ],
            'lobulated cystic lesion': [
                r'\blobulated\s+cystic\b',
                r'\blobular\s+cystic\b',
                r'\blobulated\b',
                r'\blobular\b'
            ],
            'cystic nodule': [
                r'\bcystic\s+nodule\b',
                r'\bcystic.*?nodule\b',
                r'\bnodule.*?cystic\b'
            ],
            'cystic mass': [
                r'\bcystic\s+mass\b',
                r'\bcystic.*?mass\b',
                r'\bmass.*?cystic\b'
            ],
            'branch duct IPMN': [
                r'\bbranch\s+duct\s+(?:type\s+)?IPMN\b',
                r'\b(?:IPMN|intraductal\s+papillary\s+mucinous\s+neoplasm).*?branch\s+duct\b'
            ],
            'main duct IPMN': [
                r'\bmain\s+duct\s+(?:type\s+)?IPMN\b',
                r'\b(?:IPMN|intraductal\s+papillary\s+mucinous\s+neoplasm).*?main\s+duct\b'
            ]
        }

    def detect_encoding(self, text_bytes):
        """바이트 데이터의 인코딩 탐지"""
        if not isinstance(text_bytes, bytes):
            try:
                text_bytes = text_bytes.encode('utf-8', errors='replace')
            except:
                return 'utf-8'  # 변환 실패 시 기본값

        result = chardet.detect(text_bytes)
        return result['encoding'] if result['encoding'] else 'utf-8'

    def fix_korean_encoding(self, text):
        """한글 인코딩 오류 수정 함수"""
        if not isinstance(text, str):
            return ""

        # 기본 패턴 매핑
        for wrong, correct in self.KOREAN_MAPPINGS.items():
            text = text.replace(wrong, correct)

        # 중국어 문자 처리
        text = re.sub(r'[\u4e00-\u9fff]+', ' ', text)

        # 깨진 한글 패턴 처리
        text = re.sub(r'[\?]{2,}', ' ', text)
        text = re.sub(r'\?[^\s\.,;:!\"\'\(\)\?]+', ' ', text)

        return text

    def clean_text(self, text):
        """텍스트 정제 함수"""
        if not isinstance(text, str):
            return ""

        # 한글 인코딩 오류 수정
        text = self.fix_korean_encoding(text)

        # 이스케이프 시퀀스 제거
        text = re.sub(r'_x000D_', '', text)
        text = re.sub(r'\\n', '\n', text)
        text = re.sub(r'\*x000D\*', '', text)

        # 문자열 내 남은 깨진 한글 패턴 제거
        text = re.sub(r'[\?][^\s\.,;:!\"\'\(\)]*', ' ', text)

        # 연속된 공백 제거
        text = re.sub(r'\s+', ' ', text)

        # 여러 줄바꿈을 하나로 통일
        text = re.sub(r'\n\s*\n+', '\n\n', text)

        return text.strip()

    def extract_other_organs_with_context(self, text):
        """다른 장기 언급과 관련 컨텍스트를 함께 추출"""
        if not text:
            return ""

        # 주요 복부 장기 패턴 (영어와 한글)
        organ_patterns = {
            'liver/간': re.compile(r'\bliver\b|\bhepatic\b|\b간\b', re.IGNORECASE),
            'gallbladder/담낭': re.compile(r'\bgallbladder\b|\bgb\b|\b담낭\b', re.IGNORECASE),
            'spleen/비장': re.compile(r'\bspleen\b|\bsplenic\b|\b비장\b', re.IGNORECASE),
            'kidney/신장': re.compile(r'\bkidney\b|\brenal\b|\b신장\b', re.IGNORECASE),
            'stomach/위': re.compile(r'\bstomach\b|\bgastric\b|\b위\b(?!치|험)', re.IGNORECASE),
            'colon/대장': re.compile(r'\bcolon\b|\bcolonic\b|\b대장\b', re.IGNORECASE),
            'duodenum/십이지장': re.compile(r'\bduodenum\b|\bduodenal\b|\b십이지장\b', re.IGNORECASE),
            'bile duct/담도': re.compile(r'\bbile duct\b|\bbiliary\b|\b담도\b', re.IGNORECASE),
            'CBD/총담관': re.compile(r'\bcommon bile duct\b|\bcbd\b|\b총담관\b', re.IGNORECASE)
        }

        # 관련 소견 및 병변 패턴
        finding_patterns = {
            'lesion': re.compile(r'\blesion\b|\bmass\b|\bnodule\b|\btumor\b|\bcyst\b', re.IGNORECASE),
            'polyp': re.compile(r'\bpolyp\b|\bpolypoid\b', re.IGNORECASE),
            'stone': re.compile(r'\bstone\b|\bcalculus\b|\bcalculi\b', re.IGNORECASE),
            'fatty': re.compile(r'\bfatty\b|\bsteatosis\b|\bfatty change\b', re.IGNORECASE),
            'cirrhosis': re.compile(r'\bcirrhosis\b|\bcirrhotic\b', re.IGNORECASE),
            'enlargement': re.compile(r'\benlargement\b|\benlarged\b|\bhepatosplenomegaly\b|\bhepatomegaly\b|\bsplenomegaly\b', re.IGNORECASE),
            'cystic': re.compile(r'\bcystic\b|\bsimple cyst\b', re.IGNORECASE),
            'inflammation': re.compile(r'\binflammation\b|\binflammatory\b|\b-itis\b', re.IGNORECASE),
            'adenoma': re.compile(r'\badenoma\b', re.IGNORECASE),
            '병변': re.compile(r'\b병변\b', re.IGNORECASE),
            '결절': re.compile(r'\b결절\b', re.IGNORECASE),
            '낭종': re.compile(r'\b낭종\b', re.IGNORECASE),
            '폴립': re.compile(r'\b폴립\b', re.IGNORECASE),
            '종괴': re.compile(r'\b종괴\b', re.IGNORECASE),
            '결석': re.compile(r'\b결석\b', re.IGNORECASE),
            '지방': re.compile(r'\b지방\b', re.IGNORECASE),
            '경변': re.compile(r'\b경변\b', re.IGNORECASE),
            '염증': re.compile(r'\b염증\b', re.IGNORECASE),
            '선종': re.compile(r'\b선종\b', re.IGNORECASE)
        }

        organ_findings = []

        # 텍스트를 문장으로 분리
        sentences = re.split(r'(?<=[.?!])\s+', text)

        for sentence in sentences:
            # 각 문장에서 장기 패턴 검색
            for organ_name, organ_pattern in organ_patterns.items():
                if organ_pattern.search(sentence):
                    organ_match = organ_pattern.search(sentence)

                    # 해당 문장에서 소견 패턴 검색
                    findings = []
                    for finding_name, finding_pattern in finding_patterns.items():
                        if finding_pattern.search(sentence):
                            findings.append(finding_name)

                    # 소견이 있으면 "장기: 소견" 형태로 추가
                    if findings:
                        organ_context = f"{organ_name}: {', '.join(findings)}"
                        # 앞뒤 문맥 포함
                        start = max(0, organ_match.start() - 20)
                        end = min(len(sentence), organ_match.end() + 40)
                        context = sentence[start:end].strip()
                        organ_context += f" ({context})"
                        organ_findings.append(organ_context)
                    else:
                        # 소견이 없으면 문장 일부만 추가
                        start = max(0, organ_match.start() - 10)
                        end = min(len(sentence), organ_match.end() + 25)
                        context = sentence[start:end].strip()
                        organ_findings.append(f"{organ_name} ({context})")

        # 결과가 너무 길면 잘라내기
        result = "; ".join(organ_findings)
        if len(result) > 1000:  # 최대 길이 제한
            result = result[:997] + "..."

        return result

    def advanced_extract_lesion_features(self, conclusion, report_text=None):
        """
        정교한 병변 특성 추출 함수
        - 형태학적 특징 기반 병변 분류
        - 일반 cyst와 특수 유형(IPMN, septated cyst 등) 구분
        - 원문의 병변 묘사를 최대한 보존
        """
        if not conclusion:
            return {
                'lesion_presence': 'No',
                'lesion_type': '',
                'complex_lesion_type': '',
                'multiplicity': 'No',
                'location': '',
                'size': '',
                'signal_intensity': '',
                'change': '',
                'other_organs': ''
            }

        # 결과 초기화
        features = {
            'lesion_presence': 'No',
            'lesion_type': '',
            'complex_lesion_type': '',
            'multiplicity': 'No',
            'location': '',
            'size': '',
            'signal_intensity': '',
            'change': '',
            'other_organs': ''
        }

        # 텍스트 정제
        conclusion = self.clean_text(conclusion)
        if report_text:
            report_text = self.clean_text(report_text)

        # 모든 텍스트 결합 (일부 정보는 report_text에만 있을 수 있음)
        combined_text = conclusion
        if report_text:
            combined_text += " " + report_text

        # 항목별로 분리
        items = []
        numbered_items = re.split(r'\n\s*\d+\.', conclusion)

        if len(numbered_items) <= 1:
            # 번호 매김이 없는 경우 문장 단위로 분리
            items = re.split(r'(?<=[.?!])\s+', conclusion)
        else:
            # 번호 매김이 있는 경우
            for item in numbered_items:
                if not item.strip():
                    continue
                if item == numbered_items[0] and not re.match(r'^\s*\d+\.', conclusion):
                    items.append(item.strip())
                else:
                    items.append(re.sub(r'^\s*\d+\.', '', item).strip())

        # 췌장 관련 항목만 필터링
        pancreas_items = []
        for item in items:
            if not item:
                continue
            # 췌장 관련 항목인지 확인
            if re.search(r'\b(?:pancreas|pancreatic|IPMN|p-duct|췌장)\b', item, re.IGNORECASE):
                pancreas_items.append(item)

        # 전체 텍스트에서도 췌장 관련 문장 추출
        if report_text and not pancreas_items:
            sentences = re.split(r'(?<=[.?!])\s+', report_text)
            for sentence in sentences:
                if re.search(r'\b(?:pancreas|pancreatic|IPMN|p-duct|췌장)\b', sentence, re.IGNORECASE):
                    pancreas_items.append(sentence.strip())

        # 췌장 관련 항목이 없으면 종료
        if not pancreas_items:
            return features

        # 병변 세부 유형 분류를 위한 키워드 정의
        lesion_type_keywords = {
            # IPMN과 관련 변형
            'IPMN': [
                r'\bIPMN\b',
                r'\bbranch duct type\b',
                r'\bmain duct type\b',
                r'\bintraductal papillary mucinous\b',
                r'\bpapillary mucinous\b'
            ],

            # Cystic 변형들
            'cystic lesion': [
                r'\bcystic lesion\b',
                r'\bcystic\b',
                r'\bcyst\b'
            ],

            # Dilatation
            'dilatation': [
                r'\bdilatation\b',
                r'\bdilated\b',
                r'\bdilation\b'
            ],

            # Nodule
            'nodule': [
                r'\bnodule\b',
                r'\bnodular\b'
            ],

            # Mass
            'mass': [
                r'\bmass\b'
            ],

            # Tumor
            'tumor': [
                r'\btumor\b',
                r'\bneoplasm\b',
                r'\bneoplastic\b'
            ],

            # 기타 병변
            'lesion': [
                r'\blesion\b'
            ]
        }

        # 우선순위 순서 (특이성이 높은 것부터 낮은 것 순)
        lesion_priority = [
            'IPMN',
            'tumor',
            'nodule',
            'mass',
            'dilatation',
            'cystic lesion',
            'lesion'
        ]

        # 부정 표현 패턴 (no change, no interval change는 제외)
        negative_pattern = re.compile(r'\bno\s+(?!change|interval)\b|\bnot\b|\bnormal\b|\babsence\b', re.IGNORECASE)

        # 위치, 크기, 유형 정보 추출
        locations = []
        sizes = []
        detected_lesion_types = set()
        lesion_descriptions = []

        # 각 췌장 관련 항목에서 병변 정보 추출
        for item in pancreas_items:
            # 부정 표현 확인
            is_negative = negative_pattern.search(item) and not re.search(r'no\s+(?:interval\s+)?change', item, re.IGNORECASE)
            if is_negative and not re.search(r'R/O|->|-->|rule out|differential|impression', item, re.IGNORECASE):
                continue  # 명확한 부정 표현이 있으면 건너뛰기

            # 병변 유형 추출
            for lesion_type, patterns in lesion_type_keywords.items():
                for pattern in patterns:
                    if re.search(pattern, item, re.IGNORECASE):
                        detected_lesion_types.add(lesion_type)

                        # 병변 묘사 추출
                        match = re.search(pattern, item, re.IGNORECASE)
                        if match:
                            start = max(0, match.start() - 15)
                            end = min(len(item), match.end() + 15)
                            context = item[start:end].strip()
                            context = re.sub(r'^[^\w]*', '', context)
                            context = re.sub(r'[^\w]*$', '', context)
                            lesion_descriptions.append(context)
                        break

            # 위치 정보 추출
            for location, pattern in self.LOCATION_PATTERNS.items():
                if pattern.search(item):
                    locations.append(location)

            # 크기 정보 추출 - 기본 패턴
            size_matches = self.SIZE_PATTERN.findall(item)
            if size_matches:
                sizes.extend(size_matches)

            # 추가 크기 패턴 적용
            for pattern in self.ADDITIONAL_SIZE_PATTERNS:
                more_sizes = pattern.findall(item)
                if more_sizes:
                    # 튜플로 반환되는 경우 (범위 표현 등) 처리
                    for match in more_sizes:
                        if isinstance(match, tuple):
                            sizes.extend(match)
                        else:
                            sizes.append(match)

        # 변화 정보 추출
        change_matches = self.CHANGE_PATTERN.findall(combined_text)
        if change_matches:
            # 중복 제거 및 변화 정보 조합
            unique_changes = sorted(set(change_matches))
            features['change'] = ', '.join(unique_changes)

        # 신호 강도 정보 추출
        signal_matches = self.SIGNAL_PATTERN.findall(combined_text)
        if signal_matches:
            # 중복 제거 및 신호 강도 정보 조합
            unique_signals = sorted(set(signal_matches))
            features['signal_intensity'] = ', '.join(unique_signals)

        # 병변이 감지되었으면 'lesion_presence'를 'Yes'로 설정
        if detected_lesion_types:
            features['lesion_presence'] = 'Yes'

            # 우선순위에 따라 가장 구체적인 병변 유형 선택
            for lesion_type in lesion_priority:
                if lesion_type in detected_lesion_types:
                    features['lesion_type'] = lesion_type
                    break

            # 병변 묘사가 있으면 병변 유형에 포함
            if lesion_descriptions and not features['lesion_type']:
                # 가장 긴 묘사 선택 (일반적으로 더 구체적)
                longest_description = max(lesion_descriptions, key=len)
                if len(longest_description) > 3:  # 최소 길이 확인
                    features['lesion_type'] = longest_description

        # Complex lesion type 추출
        for complex_type, patterns in self.COMPLEX_LESION_TYPES.items():
            for pattern in patterns:
                if re.search(pattern, combined_text, re.IGNORECASE):
                    features['complex_lesion_type'] = complex_type
                    break
            if features['complex_lesion_type']:
                break

        # 위치 정보 병합
        if locations:
            features['location'] = ', '.join(sorted(set(locations)))

        # 크기 정보 병합
        if sizes:
            # 중복 제거 및 숫자로 정렬
            unique_sizes = sorted(set(sizes), key=lambda x: float(x))
            features['size'] = ', '.join(unique_sizes)

        # 다발성 여부 판단
        combined_text = ' '.join(pancreas_items)

        # 다발성 관련 키워드 확인
        multiplicity_terms = [
            'multiple', 'several', 'a few', 'few', 'two', 'three', 'four', 'five',
            'many', 'numerous', 'both', 'bilateral', 'multifocal'
        ]

        # 다발성 판단
        for term in multiplicity_terms:
            if re.search(r'\b' + term + r'\b', combined_text, re.IGNORECASE):
                features['multiplicity'] = 'Yes'
                break

        # 위치가 2개 이상이면 다발성으로 간주
        if len(set(locations)) >= 2:
            features['multiplicity'] = 'Yes'

        # no change 표현이 있어도 병변 유형이 감지되면 'lesion_presence'를 'Yes'로 설정
        if re.search(r'\bno\s+(?:interval\s+)?change\b', combined_text, re.IGNORECASE) and detected_lesion_types:
            features['lesion_presence'] = 'Yes'

        # other_organs 필드 추출
        features['other_organs'] = self.extract_other_organs_with_context(combined_text)

        return features

    def process_file(self, df):
        """
        판독문 데이터를 처리하여 병변 특성을 추출하고 결과 데이터프레임을 반환
        """
        results = []

        for idx, row in df.iterrows():
            # 필수 필드 추출 (ID, 검사코드 포함)
            patient_id = row.get('변환 ID', '')  # ID 컬럼명
            exam_code = row.get('검사코드#5', '')  # 검사코드 컬럼명
            exam_date = row.get('처방일자#3', '')
            exam_type = row.get('검사명#6', '')
            report_text = row.get('검사결과(text)#7', '')
            conclusion = row.get('결론및진단#8', '')

            # 병변 특성 추출
            lesion_features = self.advanced_extract_lesion_features(conclusion, report_text)

            # 결과 통합 - ID와 검사코드를 맨 앞에 추가
            result = {
                'id': patient_id,
                'exam_code': exam_code,
                'exam_type': exam_type,
                'exam_date': exam_date,
                'lesion_presence': lesion_features.get('lesion_presence', 'No'),
                'lesion_type': lesion_features.get('lesion_type', ''),
                'complex_lesion_type': lesion_features.get('complex_lesion_type', ''),
                'multiplicity': lesion_features.get('multiplicity', 'No'),
                'location': lesion_features.get('location', ''),
                'size': lesion_features.get('size', ''),
                'signal_intensity': lesion_features.get('signal_intensity', ''),
                'change': lesion_features.get('change', ''),
                'other_organs': lesion_features.get('other_organs', ''),
                'conclusion': conclusion
            }

            results.append(result)

        return pd.DataFrame(results)


# 분석 실행 함수
def run_updated_analyzer():
    """개선된 분석기로 실행하는 함수"""
    print("판독문 데이터 파일을 업로드해주세요:")

    # 파일 업로드
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return None

    # 첫 번째 업로드된 파일 이름 가져오기
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    print(f"파일 '{file_name}' 업로드 완료 ({len(file_content)} bytes)")

    # 분석기 초기화
    analyzer = PancreaticRadiologyAnalyzer()

    # 인코딩 감지
    detected_encoding = analyzer.detect_encoding(file_content)
    print(f"감지된 인코딩: {detected_encoding}")

    # 파일 형식에 따라 데이터프레임으

In [ ]:
import re
import pandas as pd
from google.colab import files
import io
from collections import Counter
import chardet

# 기존 PancreaticRadiologyAnalyzer 클래스를 확장한 개선 버전
class ImprovedPancreaticRadiologyAnalyzer:
    def __init__(self):
        # 위치 패턴 - 한글/영어 표현 모두 포함
        self.LOCATION_PATTERNS = {
            'head': re.compile(r'\bhead\b|\b두부\b|\b췌두부\b|\b췌장\s*두부\b', re.IGNORECASE),
            'body': re.compile(r'\bbody\b|\b체부\b|\b췌체부\b|\b췌장\s*체부\b', re.IGNORECASE),
            'tail': re.compile(r'\btail\b|\b미부\b|\b췌미부\b|\b췌장\s*미부\b', re.IGNORECASE),
            'neck': re.compile(r'\bneck\b|\b경부\b|\b췌경부\b|\b췌장\s*경부\b', re.IGNORECASE),
            'uncinate process': re.compile(r'\buncinate process\b|\b구상돌기\b|\b췌장\s*구상돌기\b', re.IGNORECASE),
            'whole pancreas': re.compile(r'\bwhole pancreas\b|\b췌장\s*전체\b|\b전체\s*췌장\b', re.IGNORECASE)
        }

        # 크기 패턴 - 다양한 표현 방식 포함
        self.SIZE_PATTERN = re.compile(r'(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)', re.IGNORECASE)

        # 추가 크기 패턴
        self.ADDITIONAL_SIZE_PATTERNS = [
            # "약 X cm 크기" 형태
            re.compile(r'약\s+(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*(?:크기|size))?', re.IGNORECASE),
            # "XcmYmm 크기" 형태 (공백 없는 경우)
            re.compile(r'(\d+\.?\d*)(?:mm|cm|㎜|㎝)(?:\s*(?:크기|size))?', re.IGNORECASE),
            # 범위 표현: "X-Y cm" 형태
            re.compile(r'(\d+\.?\d*)[~\-](\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)', re.IGNORECASE),
            # 변화 표현: "Xcm에서 Ycm으로" 형태
            re.compile(r'(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*에서\s*)(\d+\.?\d*)\s*(?:mm|cm|㎜|㎝)(?:\s*으?로)', re.IGNORECASE)
        ]

        # 병변 유형별 핵심 패턴 강화

        # 1. 낭종(Cyst) 관련 핵심 패턴
        self.CYST_PATTERNS = {
            # 기본 낭종 표현
            'cyst': re.compile(r'\bcyst\b|\bcystic lesion\b|\b낭종\b|\b낭성\s*병변\b', re.IGNORECASE),

            # 시간적 분류 (렉시콘에서 발견된 중요 패턴)
            'new_cyst': re.compile(r'\bnew(?:ly)?\s*(?:develop\w+|appear\w+)?\s*cyst\b|\b새로운?\s*(?:발견된?)?\s*낭종\b', re.IGNORECASE),
            'increased_cyst': re.compile(r'\b(?:cyst|낭종).*?(?:increase[d]?|enlarge[d]?|progression)\b|\b(?:크기)?\s*(?:증가|커진)\s*(?:낭종|낭성\s*병변)\b', re.IGNORECASE),
            'decreased_cyst': re.compile(r'\b(?:cyst|낭종).*?(?:decrease[d]?|smaller|regression)\b|\b(?:크기)?\s*(?:감소|작아진)\s*(?:낭종|낭성\s*병변)\b', re.IGNORECASE),
            'stable_cyst': re.compile(r'\b(?:cyst|낭종).*?(?:stable|unchanged|no\s*change)\b|\b변화\s*없는\s*(?:낭종|낭성\s*병변)\b', re.IGNORECASE),

            # 췌관 관계 (렉시콘에서 발견된 중요 패턴)
            'main_duct_communication': re.compile(r'\bcyst.*?(?:communicat\w+|connect\w+).*?(?:main|pancreatic)\s*duct\b|\b낭종.*?(?:주|췌)관.*?(?:연결|교통)\b', re.IGNORECASE),
            'branch_duct_ipmn': re.compile(r'\bbranch\s+duct\s+(?:type\s+)?IPMN\b|\bBD-?IPMN\b|\b분지(?:형|형태)?\s*(?:IPMN|관내유두상점액종양?)\b', re.IGNORECASE),
            'main_duct_ipmn': re.compile(r'\bmain\s+duct\s+(?:type\s+)?IPMN\b|\bMD-?IPMN\b|\b주췌관(?:형|형태)?\s*(?:IPMN|관내유두상점액종양?)\b', re.IGNORECASE),
        }

        # 2. 결절(Nodule) 관련 핵심 패턴
        self.NODULE_PATTERNS = {
            # 기본 결절 표현
            'nodule': re.compile(r'\bnodule\b|\bnodular lesion\b|\b결절\b|\b결절성\s*병변\b', re.IGNORECASE),

            # MRI 신호 강도 (렉시콘에서 발견된 중요 패턴)
            't1_signal': re.compile(r'\bt1[w]?\s*(?:high|hyper|low|hypo|iso)\s*(?:signal|intensity)\b|\bt1.*?(?:고|저|동등)신호\b', re.IGNORECASE),
            't2_signal': re.compile(r'\bt2[w]?\s*(?:high|hyper|low|hypo|iso)\s*(?:signal|intensity)\b|\bt2.*?(?:고|저|동등)신호\b', re.IGNORECASE),
            'diffusion_restriction': re.compile(r'\bdiffusion\s*(?:restriction|restricted)\b|\bdwi\s*high\b|\badc\s*low\b|\b확산\s*제한\b', re.IGNORECASE),

            # 조영 증강 패턴
            'enhancement_pattern': re.compile(r'\b(?:rim|peripheral|homogeneous|heterogeneous)\s*enhancement\b|\b(?:테두리|변연부|균일|불균일)(?:한|하게)?\s*조영증강\b', re.IGNORECASE),
        }

        # 3. 종괴(Mass) 관련 핵심 패턴
        self.MASS_PATTERNS = {
            # 기본 종괴 표현
            'mass': re.compile(r'\bmass\b|\btumor\b|\b종괴\b|\b종양\b', re.IGNORECASE),

            # 크기 특성 (렉시콘에서 발견된, 종괴에서 중요한 패턴)
            'large_mass': re.compile(r'\blarge\s*(?:mass|tumor)\b|\bmassive\s*(?:mass|tumor)\b|\bhuge\s*(?:mass|tumor)\b|\b큰\s*(?:종괴|종양)\b|\b거대한\s*(?:종괴|종양)\b', re.IGNORECASE),

            # 주변 조직 영향 (렉시콘에서 발견된, 종괴에서 중요한 패턴)
            'invasion': re.compile(r'\b(?:mass|tumor|종괴|종양).*?(?:invad\w+|invasive|invasion)\b|\b(?:침윤|침습|침범)(?:하는|된)?\s*(?:종괴|종양)\b', re.IGNORECASE),
            'compression': re.compile(r'\b(?:mass|tumor|종괴|종양).*?(?:compress\w+|compression|mass effect)\b|\b(?:압박|압박효과|전위)(?:하는|된)?\s*(?:종괴|종양)\b', re.IGNORECASE),
            'vessel_involvement': re.compile(r'\b(?:mass|tumor|종괴|종양).*?(?:vessel|vascular|artery|vein).*?(?:involvement|encasement|invasion)\b|\b(?:혈관|동맥|정맥).*?(?:침범|침윤|포위)(?:하는|된)?\s*(?:종괴|종양)\b', re.IGNORECASE),
        }

        # 변화 정보 패턴 (기존 패턴 + 렉시컨 기반 확장)
        self.CHANGE_PATTERN = re.compile(
            r'\b(?:increas(?:ed?|ing)|decreas(?:ed?|ing)|no\s*change|interval\s*change|'
            r'increase[d]?\s*in\s*size|decrease[d]?\s*in\s*size|'
            r'newly\s*appear(?:ed|ing)?|disappeared?|resolution|improvement|'
            r'progress(?:ion|ive)|stable|unchanged|'
            r'변화|증가|감소|커짐|작아짐|새롭게|커진|작아진|신생|소실|'
            r'호전|악화|진행성|변화\s*없음|이전과\s*동일|이전과\s*같은|'
            r'더\s*커짐|더\s*작아짐|새로\s*생긴?|새로\s*나타남?)\b',
            re.IGNORECASE
        )

        # 신호 강도 패턴
        self.SIGNAL_PATTERN = re.compile(r'\b(?:high|low|iso)(?:\s*(?:signal|intensity|SI))\b|\bt[12][wc]?[i]?\s*(?:high|hyper|low|hypo|iso)(?:\s*(?:intense|intensity|signal))?\b|\bT[12](?:\s*강조)?(?:\s*영상)?(?:\s*(?:에서)?)?\s*(?:고|저|동등)?\s*신호(?:\s*강도)?\b', re.IGNORECASE)

        # 진단적 고려사항 관련 패턴 추가
        self.DIAGNOSTIC_PATTERNS = {
            'rule_out': re.compile(r'\br/?o\b|\brule\s*out\b|\bexclude\b|\b배제\b|\b감별\b|\br/o\b', re.IGNORECASE),
            'consistent_with': re.compile(r'\bconsistent\s*with\b|\bcompatible\s*with\b|\bsuggestive\s*of\b|\bin\s*favor\s*of\b|\b일치하는?\b|\b합당한?\b|\b부합하는?\b|\b시사하는?\b', re.IGNORECASE),
            'suspicious': re.compile(r'\bsuspicious\b|\bsuspicious\s*(?:for|of)\b|\bcannot\s*rule\s*out\b|\bcannot\s*exclude\b|\b의심(?:스러운?|되는?|되어?|됨|할수?)?\b|\b가능성\b|\b배제\s*(?:할\s*수\s*없음|어려움|곤란함|곤란|불가)\b', re.IGNORECASE),
            'differential_diagnosis': re.compile(r'\bdifferential\s*diagnos[ie]s\b|\bdifferential\s*consideration\b|\bdifferential\b|\bddx\b|\bd/dx\b|\bdd\b|\b감별\s*진단\b|\b감별\s*(?:진단)?\s*(?:필요|요함|요구됨)\b', re.IGNORECASE),
            'possibility': re.compile(r'\b(?:(?:high|low)\s*)?possibilit(?:y|ies)\b|\bcannot\s*exclude\s*(?:the)?\s*possibility\b|\b(?:(?:높은|낮은)\s*)?가능성\b|\b배제\s*(?:할\s*수\s*없는?)?\s*가능성\b', re.IGNORECASE)
        }

        # 한글 인코딩 오류 매핑
        self.KOREAN_MAPPINGS = {
            '媛?': '간',
            '鍮꾩옣': '비장',
            '?대궘': '담낭',
            '?좎옣': '신장',
            '?섑옣': '췌장',
            '?앺넻': '흉부',
            '寃고곷': '결과',
            '?뚮Т': '무우',
        }

        # Complex lesion 패턴 (기존 패턴 확장)
        self.COMPLEX_LESION_TYPES = {
            'septated cystic lesion': [
                r'\bseptat(?:ed|ion|a|ions)\s+(?:cyst|cystic)\b',
                r'\bmulti\s*sept(?:ated|ation|a|ations)\b',
                r'\bseptat(?:ed|ion|a|ions)\b',
                r'\b격벽(?:\s*있는?|성)?\s*(?:낭종|낭성|낭종성|낭성\s*병변)\b',
                r'\b중격(?:\s*있는?|성)?\s*(?:낭종|낭성|낭종성|낭성\s*병변)\b',
                r'\b다중격\s*(?:낭종|낭성|낭종성|낭성\s*병변)\b'
            ],
            'cystic nodule': [
                r'\bcystic\s+nodule\b',
                r'\bcystic.*?nodular\b',
                r'\bcystic.*?nodule\b',
                r'\bnodule.*?cystic\b',
                r'\bnodular.*?cystic\b',
                r'\b낭성\s*(?:결절|결절성|결절성?\s*병변)\b',
                r'\b낭성?\s*결절성?\s*병변\b',
                r'\b결절(?:성|성?\s*병변)?\s*낭종\b'
            ],
            'cystic mass': [
                r'\bcystic\s+mass\b',
                r'\bcystic.*?mass\b',
                r'\bmass.*?cystic\b',
                r'\b낭성\s*(?:종괴|종양|종괴성|종양성|병변|종괴성?\s*병변|종양성?\s*병변)\b',
                r'\b낭성?\s*종괴성?\s*병변\b',
                r'\b종괴(?:성|성?\s*병변)?\s*낭종\b'
            ],
            'branch duct IPMN': [
                r'\bbranch\s+(?:duct)?\s*(?:type)?\s*(?:IPMN|intraductal\s*papillary\s*mucinous\s*neoplasm)\b',
                r'\bBD\-?IPMN\b',
                r'\b분지형?\s*(?:IPMN|관내유두상점액종양?|관내유두상점액낭종?)\b'
            ],
            'main duct IPMN': [
                r'\bmain\s+(?:duct)?\s*(?:type)?\s*(?:IPMN|intraductal\s*papillary\s*mucinous\s*neoplasm)\b',
                r'\bMD\-?IPMN\b',
                r'\b주췌관형?\s*(?:IPMN|관내유두상점액종양?|관내유두상점액낭종?)\b'
            ]
        }

    def detect_encoding(self, text_bytes):
        """바이트 데이터의 인코딩 탐지"""
        if not isinstance(text_bytes, bytes):
            try:
                text_bytes = text_bytes.encode('utf-8', errors='replace')
            except:
                return 'utf-8'  # 변환 실패 시 기본값

        result = chardet.detect(text_bytes)
        return result['encoding'] if result['encoding'] else 'utf-8'

    def fix_korean_encoding(self, text):
        """한글 인코딩 오류 수정 함수"""
        if not isinstance(text, str):
            return ""

        # 기본 패턴 매핑
        for wrong, correct in self.KOREAN_MAPPINGS.items():
            text = text.replace(wrong, correct)

        # 중국어 문자 처리
        text = re.sub(r'[\u4e00-\u9fff]+', ' ', text)

        # 깨진 한글 패턴 처리
        text = re.sub(r'[\?]{2,}', ' ', text)
        text = re.sub(r'\?[^\s\.,;:!\"\'\(\)\?]+', ' ', text)

        return text

    def clean_text(self, text):
        """텍스트 정제 함수"""
        if not isinstance(text, str):
            return ""

        # 한글 인코딩 오류 수정
        text = self.fix_korean_encoding(text)

        # 이스케이프 시퀀스 제거
        text = re.sub(r'_x000D_', '', text)
        text = re.sub(r'\\n', '\n', text)
        text = re.sub(r'\*x000D\*', '', text)

        # 문자열 내 남은 깨진 한글 패턴 제거
        text = re.sub(r'[\?][^\s\.,;:!\"\'\(\)]*', ' ', text)

        # 연속된 공백 제거
        text = re.sub(r'\s+', ' ', text)

        # 여러 줄바꿈을 하나로 통일
        text = re.sub(r'\n\s*\n+', '\n\n', text)

        return text.strip()

    def extract_other_organs_with_context(self, text):
        """다른 장기 언급과 관련 컨텍스트를 함께 추출"""
        if not text:
            return ""

        # 주요 복부 장기 패턴 (영어와 한글)
        organ_patterns = {
            'liver/간': re.compile(r'\bliver\b|\bhepatic\b|\b간\b', re.IGNORECASE),
            'gallbladder/담낭': re.compile(r'\bgallbladder\b|\bgb\b|\b담낭\b', re.IGNORECASE),
            'spleen/비장': re.compile(r'\bspleen\b|\bsplenic\b|\b비장\b', re.IGNORECASE),
            'kidney/신장': re.compile(r'\bkidney\b|\brenal\b|\b신장\b', re.IGNORECASE),
            'stomach/위': re.compile(r'\bstomach\b|\bgastric\b|\b위\b(?!치|험)', re.IGNORECASE),
            'colon/대장': re.compile(r'\bcolon\b|\bcolonic\b|\b대장\b', re.IGNORECASE),
            'duodenum/십이지장': re.compile(r'\bduodenum\b|\bduodenal\b|\b십이지장\b', re.IGNORECASE),
            'bile duct/담도': re.compile(r'\bbile duct\b|\bbiliary\b|\b담도\b', re.IGNORECASE),
            'CBD/총담관': re.compile(r'\bcommon bile duct\b|\bcbd\b|\b총담관\b', re.IGNORECASE)
        }

        # 관련 소견 및 병변 패턴
        finding_patterns = {
            'lesion': re.compile(r'\blesion\b|\bmass\b|\bnodule\b|\btumor\b|\bcyst\b', re.IGNORECASE),
            'polyp': re.compile(r'\bpolyp\b|\bpolypoid\b', re.IGNORECASE),
            'stone': re.compile(r'\bstone\b|\bcalculus\b|\bcalculi\b', re.IGNORECASE),
            'fatty': re.compile(r'\bfatty\b|\bsteatosis\b|\bfatty change\b', re.IGNORECASE),
            'cirrhosis': re.compile(r'\bcirrhosis\b|\bcirrhotic\b', re.IGNORECASE),
            'enlargement': re.compile(r'\benlargement\b|\benlarged\b|\bhepatosplenomegaly\b|\bhepatomegaly\b|\bsplenomegaly\b', re.IGNORECASE),
            'cystic': re.compile(r'\bcystic\b|\bsimple cyst\b', re.IGNORECASE),
            'inflammation': re.compile(r'\binflammation\b|\binflammatory\b|\b-itis\b', re.IGNORECASE),
            'adenoma': re.compile(r'\badenoma\b', re.IGNORECASE),
            '병변': re.compile(r'\b병변\b', re.IGNORECASE),
            '결절': re.compile(r'\b결절\b', re.IGNORECASE),
            '낭종': re.compile(r'\b낭종\b', re.IGNORECASE),
            '폴립': re.compile(r'\b폴립\b', re.IGNORECASE),
            '종괴': re.compile(r'\b종괴\b', re.IGNORECASE),
            '결석': re.compile(r'\b결석\b', re.IGNORECASE),
            '지방': re.compile(r'\b지방\b', re.IGNORECASE),
            '경변': re.compile(r'\b경변\b', re.IGNORECASE),
            '염증': re.compile(r'\b염증\b', re.IGNORECASE),
            '선종': re.compile(r'\b선종\b', re.IGNORECASE)
        }

        organ_findings = []

        # 텍스트를 문장으로 분리
        sentences = re.split(r'(?<=[.?!])\s+', text)

        for sentence in sentences:
            # 각 문장에서 장기 패턴 검색
            for organ_name, organ_pattern in organ_patterns.items():
                if organ_pattern.search(sentence):
                    organ_match = organ_pattern.search(sentence)

                    # 해당 문장에서 소견 패턴 검색
                    findings = []
                    for finding_name, finding_pattern in finding_patterns.items():
                        if finding_pattern.search(sentence):
                            findings.append(finding_name)

                    # 소견이 있으면 "장기: 소견" 형태로 추가
                    if findings:
                        organ_context = f"{organ_name}: {', '.join(findings)}"
                        # 앞뒤 문맥 포함
                        start = max(0, organ_match.start() - 20)
                        end = min(len(sentence), organ_match.end() + 40)
                        context = sentence[start:end].strip()
                        organ_context += f" ({context})"
                        organ_findings.append(organ_context)
                    else:
                        # 소견이 없으면 문장 일부만 추가
                        start = max(0, organ_match.start() - 10)
                        end = min(len(sentence), organ_match.end() + 25)
                        context = sentence[start:end].strip()
                        organ_findings.append(f"{organ_name} ({context})")

        # 결과가 너무 길면 잘라내기
        result = "; ".join(organ_findings)
        if len(result) > 1000:  # 최대 길이 제한
            result = result[:997] + "..."

        return result

    def extract_lesion_features(self, conclusion, report_text=None):
        """
        렉시콘 기반으로 개선된 병변 특성 추출 함수
        """
        # 결과 초기화
        features = {
            'lesion_presence': 'No',
            'lesion_type': '',
            'complex_lesion_type': '',
            'multiplicity': 'No',
            'location': '',
            'size': '',
            'signal_intensity': '',
            'change': '',
            'other_organs': '',
            'temporal_classification': '',  # 시간적 분류 (낭종 특화)
            'duct_relation': '',           # 췌관 관계 (낭종 특화)
            'mri_signal': '',              # MRI 신호 특성 (결절 특화)
            'enhancement_pattern': '',     # 조영 증강 패턴 (결절 특화)
            'size_description': '',        # 크기 특성 (종괴 특화)
            'tissue_effect': ''            # 주변 조직 영향 (종괴 특화)
        }

        # 텍스트 정제
        conclusion = self.clean_text(conclusion)
        if report_text:
            report_text = self.clean_text(report_text)

        # 모든 텍스트 결합
        combined_text = conclusion
        if report_text:
            combined_text += " " + report_text

        # 항목별로 분리
        items = []
        numbered_items = re.split(r'\n\s*\d+\.', conclusion)

        if len(numbered_items) <= 1:
            # 번호 매김이 없는 경우 문장 단위로 분리
            items = re.split(r'(?<=[.?!])\s+', conclusion)
        else:
            # 번호 매김이 있는 경우
            for item in numbered_items:
                if not item.strip():
                    continue
                if item == numbered_items[0] and not re.match(r'^\s*\d+\.', conclusion):
                    items.append(item.strip())
                else:
                    items.append(re.sub(r'^\s*\d+\.', '', item).strip())

        # 췌장 관련 항목만 필터링
        pancreas_items = []
        for item in items:
            if not item:
                continue
            # 췌장 관련 항목인지 확인
            if re.search(r'\b(?:pancreas|pancreatic|IPMN|p-duct|췌장)\b', item, re.IGNORECASE):
                pancreas_items.append(item)

        # 전체 텍스트에서도 췌장 관련 문장 추출
        if report_text and not pancreas_items:
            sentences = re.split(r'(?<=[.?!])\s+', report_text)
            for sentence in sentences:
                if re.search(r'\b(?:pancreas|pancreatic|IPMN|p-duct|췌장)\b', sentence, re.IGNORECASE):
                    pancreas_items.append(sentence.strip())

        # 췌장 관련 항목이 없으면 종료
        if not pancreas_items:
            return features

        # 부정 표현 패턴 (no change, no interval change는 제외)
        negative_pattern = re.compile(r'\bno\s+(?!change|interval)\b|\bnot\b|\bnormal\b|\babsence\b', re.IGNORECASE)

        # 병변 유형 감지
        detected_lesion_types = {
            'cyst': False,
            'nodule': False,
            'mass': False
        }

        # 모든 췌장 관련 항목 검사
        for item in pancreas_items:
            # 부정 표현 확인 (단, 'no change'나 'rule out' 등은 제외)
            is_negative = negative_pattern.search(item) and not re

In [ ]:
def run_with_improved_analyzer():
    print("렉시콘 기반 개선된 판독문 분석기를 실행합니다:")

    # 파일 업로드
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return None

    # 첫 번째 업로드된 파일 이름 가져오기
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    print(f"파일 '{file_name}' 업로드 완료 ({len(file_content)} bytes)")

    # 엑셀/CSV 파일 읽기
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        df = pd.read_excel(io.BytesIO(file_content))
    else:
        df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8', errors='replace')

    print(f"파일에서 {len(df)} 행의 데이터를 읽었습니다.")
    print("데이터프레임 컬럼:", df.columns.tolist())

    # 개선된 분석기 초기화
    analyzer = ImprovedPancreaticRadiologyAnalyzer()

    # 결과 처리
    results = []

    for idx, row in df.iterrows():
        # ID 및 필수 필드 안전하게 추출
        patient_id = str(row.get('변환 ID', '')) if '변환 ID' in df.columns else ''
        exam_code = str(row.get('검사코드#5', '')) if '검사코드#5' in df.columns else ''
        exam_date = str(row.get('처방일자#3', '')) if '처방일자#3' in df.columns else ''
        exam_type = str(row.get('검사명#6', '')) if '검사명#6' in df.columns else ''
        report_text = str(row.get('검사결과(text)#7', '')) if '검사결과(text)#7' in df.columns else ''
        conclusion = str(row.get('결론및진단#8', '')) if '결론및진단#8' in df.columns else ''

        # 병변 특성 추출
        lesion_features = analyzer.extract_lesion_features(conclusion, report_text)

        # 결과 통합
        result = {
            'id': patient_id,
            'exam_code': exam_code,
            'exam_type': exam_type,
            'exam_date': exam_date,
            'lesion_presence': lesion_features.get('lesion_presence', 'No'),
            'lesion_type': lesion_features.get('lesion_type', ''),
            'complex_lesion_type': lesion_features.get('complex_lesion_type', ''),
            'multiplicity': lesion_features.get('multiplicity', 'No'),
            'location': lesion_features.get('location', ''),
            'size': lesion_features.get('size', ''),
            'signal_intensity': lesion_features.get('signal_intensity', ''),
            'change': lesion_features.get('change', ''),
            'temporal_classification': lesion_features.get('temporal_classification', ''),
            'duct_relation': lesion_features.get('duct_relation', ''),
            'mri_signal': lesion_features.get('mri_signal', ''),
            'enhancement_pattern': lesion_features.get('enhancement_pattern', ''),
            'size_description': lesion_features.get('size_description', ''),
            'tissue_effect': lesion_features.get('tissue_effect', ''),
            'other_organs': lesion_features.get('other_organs', ''),
            'conclusion': conclusion
        }

        results.append(result)

        # 진행상황 표시
        if (idx + 1) % 10 == 0 or idx == len(df) - 1:
            print(f"처리 중: {idx + 1}/{len(df)} 완료")

    # 결과 데이터프레임 생성
    results_df = pd.DataFrame(results)

    # 결과를 CSV 파일로 저장
    output_filename = f"improved_analysis_{file_name.split('.')[0]}.csv"
    results_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"분석 결과가 '{output_filename}' 파일로 저장되었습니다.")

    # 결과 미리보기 출력
    print("\n분석 결과 미리보기:")
    display(results_df.head())

    return results_df

# 이 함수를 실행하면 개선된 분석기가 사용됩니다
# run_with_improved_analyzer()

In [ ]:
def run_with_improved_analyzer():
    print("렉시콘 기반 개선된 판독문 분석기를 실행합니다:")

    # 파일 업로드
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return None

    # 첫 번째 업로드된 파일 이름 가져오기
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    print(f"파일 '{file_name}' 업로드 완료 ({len(file_content)} bytes)")

    # 파일 형식에 따라 데이터프레임 생성
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        df = pd.read_excel(io.BytesIO(file_content))
    else:
        try:
            df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(io.BytesIO(file_content), encoding='cp949')

    # 개선된 분석기 초기화
    analyzer = ImprovedPancreaticRadiologyAnalyzer()

    # 결과 처리
    results = []

    for idx, row in df.iterrows():
        # ID 및 필수 필드 안전하게 추출
        patient_id = str(row.get('변환 ID', '')) if '변환 ID' in df.columns else ''
        exam_code = str(row.get('검사코드#5', '')) if '검사코드#5' in df.columns else ''
        exam_date = str(row.get('처방일자#3', '')) if '처방일자#3' in df.columns else ''
        exam_type = str(row.get('검사명#6', '')) if '검사명#6' in df.columns else ''
        report_text = str(row.get('검사결과(text)#7', '')) if '검사결과(text)#7' in df.columns else ''
        conclusion = str(row.get('결론및진단#8', '')) if '결론및진단#8' in df.columns else ''

        # 병변 특성 추출
        try:
            lesion_features = analyzer.extract_lesion_features(conclusion, report_text)
            if lesion_features is None:
                # None이 반환되면 빈 딕셔너리로 대체
                print(f"행 {idx}: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.")
                lesion_features = {}
        except Exception as e:
            print(f"행 {idx} 처리 중 오류 발생: {e}")
            lesion_features = {}

        # 결과 통합
        result = {
            'id': patient_id,
            'exam_code': exam_code,
            'exam_type': exam_type,
            'exam_date': exam_date,
            'lesion_presence': lesion_features.get('lesion_presence', 'No'),
            'lesion_type': lesion_features.get('lesion_type', ''),
            'complex_lesion_type': lesion_features.get('complex_lesion_type', ''),
            'multiplicity': lesion_features.get('multiplicity', 'No'),
            'location': lesion_features.get('location', ''),
            'size': lesion_features.get('size', ''),
            'signal_intensity': lesion_features.get('signal_intensity', ''),
            'change': lesion_features.get('change', ''),
            'temporal_classification': lesion_features.get('temporal_classification', ''),
            'duct_relation': lesion_features.get('duct_relation', ''),
            'mri_signal': lesion_features.get('mri_signal', ''),
            'enhancement_pattern': lesion_features.get('enhancement_pattern', ''),
            'size_description': lesion_features.get('size_description', ''),
            'tissue_effect': lesion_features.get('tissue_effect', ''),
            'other_organs': lesion_features.get('other_organs', ''),
            'conclusion': conclusion
        }

        results.append(result)

        # 진행상황 표시
        if (idx + 1) % 10 == 0 or idx == len(df) - 1:
            print(f"처리 중: {idx + 1}/{len(df)} 완료")

    # 결과 데이터프레임 생성
    results_df = pd.DataFrame(results)

    # 결과를 CSV 파일로 저장
    output_filename = f"improved_analysis_{file_name.split('.')[0]}.csv"
    results_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"분석 결과가 '{output_filename}' 파일로 저장되었습니다.")

    return results_df

# 이 함수를 실행하면 개선된 분석기가 사용됩니다
run_with_improved_analyzer()

렉시콘 기반 개선된 판독문 분석기를 실행합니다:


Saving pancreas_1000data.csv to pancreas_1000data (4).csv
파일 'pancreas_1000data (4).csv' 업로드 완료 (1238798 bytes)
행 0: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 1: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 2: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 3: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 4: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 5: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 6: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 7: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 8: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
처리 중: 10/1000 완료
행 10: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 11: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 12: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 13: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 14: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 15: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 16: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 17: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 18: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 19: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
처리 중: 20/1000 완료
행 20: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 21: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 22: 특성 추출 결과가 None입니다. 기본값으로 대체합니다.
행 24: 특성 추출 결과가 None입니다. 기본

,id,exam_code,exam_type,exam_date,lesion_presence,lesion_type,complex_lesion_type,multiplicity,location,size,signal_intensity,change,temporal_classification,duct_relation,mri_signal,enhancement_pattern,size_description,tissue_effect,other_organs,conclusion
0,B47301DE00A340,RM2040,PANCREAS MRI (PRE- & POST-CONTRAST) + DIFFUSION,2022-11-19 00:00:00,No,,,No,,,,,,,,,,,,1. Tiny cystic lesion in the pancreas head.\r\...
1,B47301DE00A340,RM2040,PANCREAS MRI (PRE- & POST-CONTRAST) + DIFFUSION,2023-11-28 00:00:00,No,,,No,,,,,,,,,,,,1. A few tiny cystic lesions in the pancreas h...
2,B47301DE00A340,RM2040,PANCREAS MRI (PRE- & POST-CONTRAST) + DIFFUSION,2025-02-13 00:00:00,No,,,No,,,,,,,,,,,,"1. No change of two tiny pancreatic cysts, hea..."
3,4623B1DE00A34F,RC31202,ABDOMEN AND PELVIS CT (POST-CONTRAST),2020-06-27 00:00:00,No,,,No,,,,,,,,,,,,Normal abdominal + pelvis CT.
4,70CEA3DE00A398,RM2040,PANCREAS MRI (PRE- & POST-CONTRAST) + DIFFUSION,2021-09-13 00:00:00,No,,,No,,,,,,,,,,,,1. Two lobular septated cystic lesions in panc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,378355DE017358,RM2040,PANCREAS MRI (PRE- & POST-CONTRAST) + DIFFUSION,2024-09-05 00:00:00,No,,,No,,,,,,,,,,,,"1. R/O Tiny cystic lesion, uncinate process of..."
996,9FC579DE0173C4,RC31202,ABDOMEN AND PELVIS CT (POST-CONTRAST),2022-02-10 00:00:00,No,,,No,,,,,,,,,,,,1. S/P Cholecystectomy.\r\n\r\n2. Tiny low att...
997,4842A3DE017455,RC20301,LIVER AND PELVIS CT (POST-CONTRAST),2020-03-27 00:00:00,No,,,No,,,,,,,,,,,,1. No evidence of HCC.\r\n2. Several tiny beni...
998,96F60BDE01748A,RC31202,ABDOMEN AND PELVIS CT (POST-CONTRAST),2024-09-10 00:00:00,No,,,No,,,,,,,,,,,,Very tiny low attenuation lesion in the right ...
